In [2]:
import torch
import numpy as np
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import seed_everything
seed_everything(0)


/home/seongha/anaconda3/envs/LTML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


coco

In [3]:
path_csv = '../data/coco'

from collections import defaultdict, Counter
class_num = defaultdict(int)
with open(path_csv + '/data/train_anno.json') as f:
  adj = np.zeros((80,80))
  import json
  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    for l in each['labels']:
      class_num[l] += 1

  nums = gt_labels.sum(axis=0)
  adj = []
  for i,col in enumerate(gt_labels.T):
    if i in [34]:
      print(i)
      selected = gt_labels[np.isin(col, [1.0]), :]
      nonzero_cnt = (selected != 0).sum(1)
      cnter = Counter(nonzero_cnt)
      print(cnter)
    cond_prob = gt_labels[np.isin(col,[1.0]),:].sum(axis=0)
    cond_prob[i] = 0
    adj.append(cond_prob)
    # print(adj[-1])
  nums = nums.tolist()
  nums.sort()
  nums.reverse()
  # nums = reversed(nums)
  print(max(nums), min(nums))
  di={'adj': np.asarray(adj), "nums": np.asarray(nums)}
  class_di = {k: v for k, v in sorted(class_num.items(), key=lambda item: item[1], reverse=True)} #sorted
print(class_di.keys(), class_di.values())

82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
34
Counter({1: 1234, 2: 407, 3: 116, 4: 27, 5: 8, 8: 3, 7: 2, 6: 1})
45174.0 128.0
dict_keys([49, 22, 18, 27, 26, 13, 14, 36, 74, 2, 8, 12, 21, 57, 75, 24, 23, 41, 51, 28, 62, 72, 20, 16, 76, 68, 7, 32, 77, 58, 61, 42, 73, 44, 67, 65, 70, 9, 0, 10, 59, 50, 53, 11, 17, 37, 47, 5, 4, 34, 78, 52, 54, 64, 40, 3, 30, 33, 66, 39, 25, 15, 79, 45, 46, 63, 31, 19, 1, 60, 56, 43, 29, 38, 71, 55, 6, 48, 69, 35]) dict_values([45174, 8950, 8606, 8378, 6518, 5968, 5028, 4861, 4321, 3924, 3844, 3734, 3322, 3291, 3191, 3170, 3159, 3097, 3084, 3041, 2986, 2893, 2818, 2791, 2749, 2667, 2539, 2537, 2530, 2511, 2493, 2475, 2464, 2442, 2368, 2343, 2317, 2287, 2243, 2241, 2209, 2202, 2180, 2098, 2080, 2068, 2003, 1884, 1804, 1798, 1771, 1671, 1645, 1631, 1625, 1618, 1518, 1511, 1510, 1471, 1389, 1340, 1324, 1290, 1216, 1214, 1205, 1186, 1171, 1170, 1105, 1089, 1062, 821, 700, 673, 668, 481, 151, 128])


In [16]:
test_dataset = COCO2014('../data/coco', phase='val', inp_name='../data/coco/coco_glove_word2vec.pkl')
# train_dataset = Voc2007Classification('data/voc', 'trainval', inp_name='data/voc/voc_glove_word2vec.pkl', LT=True)
# test_dataset = Voc2007Classification('data/voc', 'test', inp_name='data/voc/voc_glove_word2vec.pkl')
# train_dataset = COCO2014('data/coco', phase='train', inp_name='data/coco/coco_glove_word2vec.pkl')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
test_dataset.transform = transforms.Compose([
                MultiScaleCrop(224, scales=(1.0, 0.875, 0.75, 0.66, 0.5), max_distort=2),
                transforms.ToTensor(),
                normalize,
            ])

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)
from util import AveragePrecisionMeter
AP = AveragePrecisionMeter(difficult_examples=False)

[dataset] Done!
[annotation] Done!
[json] Done!


In [14]:

from models import *


m_li = [base_resnet50(80, pretrained=True), base_vit(80, image_size=224, pretrained=True), base_swin(80, image_size=224, pretrained=True)]
p_li = ['/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_count_resnet50-4-4-0_resnet50_base_best.pth.tar' ,\
  '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_count_vit-4-4-0_vit_base_best.pth.tar', \
  '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_count_swin-4-4-0_swin_base_best.pth.tar', \
  ]

def get_model(index):
  path = p_li[index]
  model = m_li[index]
  di = torch.load(path)
  print(di['best_score'])
  print(di.keys())
  model.load_state_dict(di['state_dict'])
  return model


Linear(in_features=768, out_features=80, bias=True)


In [17]:
model = get_model(0)
for i, (input, target) in tqdm(enumerate(test_loader)):
  img, path, inp = input
  target[target == 0] = 1
  target[target == -1] = 0
  feat_Var = torch.autograd.Variable(img).float().detach()
  
  # output = model(feat_Var, None).detach()
  output = model(feat_Var, None).detach()
  # print(output.requires_grad, target.requires_grad)
  AP.add(output, target)
map = 100 * AP.value().mean()
print(100 * AP.value())
ap_li = 100 * AP.value()

{'mAP': tensor(62.4261), 'OF1': 0.6323788284730504, 'CF1': 0.5572570761842324}
dict_keys(['epoch', 'arch', 'state_dict', 'best_score'])


380it [27:35,  4.36s/it]


KeyboardInterrupt: 

In [ ]:
label_li = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
       'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person',
       'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']#voc
label_li = list(class_di.keys()) #coco

#classifier
ap_li = [75.4188, 76.3768, 66.3551, 79.1177, 45.1441, 62.4649, 83.6811, 85.9619,
        58.1541, 48.6313, 67.8912, 79.3836, 79.2103, 69.3254, 88.5846, 50.0462,
        26.8350, 57.0530, 75.6681, 71.1823]#FC
ap_li2 = [77.9105, 76.8534, 71.2095, 80.9140, 45.2674, 63.6200, 84.0961, 86.4019,
        59.8740, 52.9656, 68.0257, 80.5327, 79.3948, 71.0906, 88.7570, 49.5162,
        33.0532, 55.2915, 76.8210, 70.2609]#Transformer
ap_li_gcn = [44.9041, 51.7194, 41.8896, 52.5085, 31.0836, 42.8794, 75.6171, 61.9646,
        39.8540, 26.9209, 45.8847, 50.5602, 48.6032, 48.8849, 78.4147, 37.8425,
        15.7030, 46.4849, 45.6828, 53.3943]#gcn
ap_li_tc = [52.6595, 59.2412, 48.0880, 60.5390, 34.5672, 49.0508, 77.6743, 70.0045,
        42.0592, 32.3638, 49.0484, 57.0762, 55.9363, 55.3863, 81.9262, 41.5481,
        18.1342, 48.6349, 52.5640, 58.6664]#TC

#backbone;
ap_li_resnet50_fc_coco = [44.8806, 18.7181,  4.2505, 26.0276,  1.8891,  2.0793, 55.0725, 29.3584,
        21.6527, 13.7635, 34.6959, 50.7117,  4.5387,  9.8258, 10.3550, 42.5709,
        39.2878, 19.0187, 29.4253, 10.1510, 57.6950, 11.8762, 13.9065, 42.1499,
         4.6982, 33.9326, 10.8716, 15.9627, 32.3844, 19.4244, 61.4208, 29.5272,
         3.1157,  3.6808, 75.4917,  0.1942,  5.2710, 19.7860, 15.4992, 16.0211,
        15.0432,  3.9782, 21.1419,  8.8907, 30.8293,  1.9951, 32.1277, 21.3268,
        20.2985, 83.6436, 46.0174,  5.8537, 20.1276,  5.9066, 20.2521, 12.6364,
        54.7259, 21.3757,  3.1344,  6.6266,  3.5166,  3.2945,  3.5212, 31.7686,
        15.5602, 11.4469, 40.4510,  2.9287, 17.3079,  0.1819, 36.1461,  2.6347,
        30.3691, 31.7924, 28.6856, 21.0613, 17.1495, 20.7845,  3.5991, 73.5261]

In [ ]:
def sort_by_class_size(ap_li):
  Sorted_Ap = []
  for k, v in class_di.items():
        idx = label_li.index(k)
        print("{}, {}, {}".format(k, v, ap_li[idx]))
        Sorted_Ap.append( ap_li[idx])

  return Sorted_Ap

In [ ]:
name_map = {0: "resnet50", 1: "vit", 2: "swin"}
with open("AP_{}_fc.txt".format(name_map[0]), "w") as f:
  f.write (str(sort_by_class_size(ap_li)))

### class wise ap

In [ ]:
data = {"class_index": list(class_di.keys()),
"class_size": list(class_di.values()),
"AP": ap_li}

### label distribution csv